In [15]:
import numpy as np
import pandas as pd
import csv
import json
import time
import os

from datetime import date, datetime, timedelta

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup

In [2]:
base_url = 'https://www.booking.com/'

mac_path = '/Users/admin/Downloads/chromedriver'
comcenter_path = r'C:/Users/6142923026/Downloads/chromedriver.exe'
home_path = 'E:/Downloads/Programs/chromedriver_win32/chromedriver.exe'

# Identify check-in, check-out date

In [3]:
class chk:
    def __init__(self, cin, cout):
        self.cin = cin
        self.cout = cout

In [7]:
def chk_inout():
    
    chk_list = []
    N_DAYS = 3
    STR_LEN = len('2019-02-09')
    
    t_date = datetime.now()
    
    for i in range(1,1+N_DAYS):
        
        chkin = (t_date + timedelta(days=i))
        chkout = (t_date + timedelta(days=i+1))
        
        chkin = chkin.isoformat()[:10]
        chkout = chkout.isoformat()[:10]
        
        chk_list.append(chk(chkin, chkout))

    return chk_list

In [8]:
chk_list = chk_inout()

for c in chk_list:
    print(c.cin, c.cout)

2019-03-19 2019-03-20
2019-03-20 2019-03-21
2019-03-21 2019-03-22


# Get search result urls

In [9]:
def get_search_url(chk_list):
    
    s_results = []
    
    for chk in chk_list:

        driver = webdriver.Chrome(home_path)
        driver.get(base_url)

        assert "Booking.com" in driver.title
        
        chkin = chk.cin
        chkout = chk.cout

        time.sleep(3)

        elem = driver.find_element_by_name("ss")

        elem.clear()
        elem.send_keys("Bangkok")

        calendar_xpath = "//div[@class='xp__dates-inner xp__dates__checkin']"
        date_1_xpath = "//td[@class='bui-calendar__date' and @data-date='" + chkin + "']"
        date_2_xpath = "//td[@class='bui-calendar__date' and @data-date='" + chkout + "']"

        chkin_elem = driver.find_element_by_xpath(calendar_xpath).click()
        time.sleep(2)

        chkin_date_elem = driver.find_element_by_xpath(date_1_xpath).click()
        time.sleep(1)

        chkout_date_elem = driver.find_element_by_xpath(date_2_xpath).click()
        elem.send_keys(Keys.RETURN)

        time.sleep(3)
        driver.implicitly_wait(2)

        all_pages_xpath = "//li[@class='bui-pagination__item sr_pagination_item']"
        all_pages = driver.find_elements_by_xpath(all_pages_xpath)
#         print(all_pages)
        last_page = int(all_pages[-1].text)
#         print(last_page)
        
        hname_path = "//a[@class='hotel_name_link url']"
        hname_list = driver.find_elements_by_xpath(hname_path)
        hlink = [ele.get_attribute('href') for ele in hname_list]
        
        if(len(s_results)==0):
            s_results = hlink
        else:
            s_results = np.hstack((s_results, hlink))
        
        for i in range(2,last_page):
            print(i)
            next_page_btn_xpath = "//li[@class='bui-pagination__item bui-pagination__next-arrow']"
            next_page_btn = driver.find_element_by_xpath(next_page_btn_xpath).click()
            
            time.sleep(5)
            driver.implicitly_wait(2)

            hname_list = driver.find_elements_by_xpath(hname_path)
            hlink = [ele.get_attribute('href') for ele in hname_list]

            if(len(s_results)==0):
                s_results = hlink
            else:
                s_results = np.hstack((s_results, hlink))
            
    return s_results

In [10]:
res_url = get_search_url(chk_list)

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


## Store urls in txt file

In [11]:
with open('booking-urls.txt','w') as f:
    for url in res_url:
        f.write(url + '\n')
    f.close()

# Get information from each hotel

In [12]:
def get_hotel_table(soup, i):

    h_dict = dict(hid = [], 
                  hname = [], 
                  hlocation = [], 
                  hstars = [], 
                  hratings = [],
                  hnearby = [],
                  hfacilities = [],
                  hcheckin = [],
                  hcheckout = []
                 )

    hotel_name = soup.find('h2', class_='hp__hotel-name').getText().strip('\n').split('\n')[1]
    hotel_stars = soup.find(class_='bk-icon-wrapper bk-icon-stars star_track')
    if(hotel_stars):
        hstars = hotel_stars.getText().strip('\n')
    else:
        hstars = None

    hotel_loc = soup.find(class_='hp_address_subtitle')
    if(hotel_loc):
        hlocation = hotel_loc.getText().strip('\n')
    else:
        hlocation = None

    review_score = soup.find(class_='bui-review-score')
    if(review_score):
        score_string = review_score.getText().split(' ')
        ratings = [s for s in score_string if(s!='')][0]
    else:
        ratings = ''

    nearby_str = ''
    
    nearby_blk = soup.find(class_='property_page_surroundings_block')
    if(nearby_blk):
        nearby_pl = nearby_blk.find_all(class_='bui-list__item')

        for p in nearby_pl:
            pl_ = p.getText().replace('\n',';')
            near_pl_list = pl_.split(';')
            near_pl_list = [pp for pp in near_pl_list if(pp!='')]
            places = near_pl_list[0]
            distance = near_pl_list[-1]
            p_d = ' '.join([places, distance]) + '; '

            nearby_str = ''.join([nearby_str, p_d])

    facil_blk = soup.find(class_='hp_desc_important_facilities').getText().replace('\n',';')
    if(facil_blk):
        facility_list = facil_blk.split(';')
        facility_list = [f for f in facility_list if(f!='')][1:]

        facil = '; '.join(facility_list)
    else:
        facil = ''

    chkindate = soup.find('div', {'data-placeholder':'Check-in Date'}).getText().replace('\n','')
    chkoutdate = soup.find('div', {'data-placeholder':'Check-out Date'}).getText().replace('\n','')

    h_dict['hid'] = i
    h_dict['hname'].append(hotel_name)
    h_dict['hstars'].append(hstars)
    h_dict['hlocation'].append(hlocation)
    h_dict['hratings'].append(ratings)
    h_dict['hnearby'].append(nearby_str)
    h_dict['hfacilities'].append(facil)

    h_dict['hcheckin'].append(chkindate)
    h_dict['hcheckout'].append(chkoutdate)
    
    df_hotel = pd.DataFrame.from_dict(h_dict)

    return df_hotel

# Get rooms information

In [13]:
def get_room_table(soup, i):
    r_dict = dict(hid = [],
                  roomName = [],
                  bedType = [],
                  capacity = [],
                  rackPrice = [],
                  price = [],
                  breakfast = [],
                  features = [])

    r_dict['hid'] = i
    roomtypes = soup.find_all(class_='hprt-roomtype-icon-link')
    
    noneText = None

    if(roomtypes):
        for r in roomtypes:
            if(r):
                room_name = r.getText().strip('\n')
                r_dict['roomName'].append(room_name)

                r_parent = r.parent.parent.parent.parent.parent

                r_capacity = r_parent.find(class_='hprt-table-cell-occupancy')
                if(r_capacity):
                    r_cap = r_capacity.getText().strip('\n')
                    r_cap = r_cap[-1]
                else:
                    r_cap = noneText
                r_dict['capacity'].append(r_cap)
                
                r_bedtype = r_parent.find(class_='bed-types-wrapper')
                if(r_bedtype):
                    r_btype = r_bedtype.getText().replace('\n',';').split(';')
                    btype_list = [bt for bt in r_btype if(bt!='')]
                    btype = '; '.join(btype_list)
                else:
                    btype = noneText
                r_dict['bedType'].append(btype)

                r_features = r_parent.find(class_='hprt-facilities-block')
                if(r_features):
                    feature_list = r_features.getText().replace('\n',';').split(';')
                    f_list = [f.strip(' •') for f in feature_list if(f!='')]
                    features = '; '.join(f_list)
                else:
                    features = noneText
                r_dict['features'].append(features)

                rack_prices = []
                prices = []

                price_block = r_parent.findNext('td', class_='hprt-table-cell-price')
                if(price_block):

                    price = price_block.find(class_='hprt-price-price')
                    pr = price.getText().replace('\n','')
                    prices.append(pr)

                    rack_rate_price = price_block.find(class_='hp-blue-sans-rack-rate')
                    if(rack_rate_price):
                        rack_pr = rack_rate_price.getText().replace('\n','')
                    else:
                        rack_pr = pr
                    rack_prices.append(rack_pr)

                    bf_icon = r_parent.find(class_='hprt-conditions')
                    if(bf_icon):
                        bf_txt_list = bf_icon.getText().replace('\n',';').split(';')
                        bf_txt = '; '.join([b for b in bf_txt_list if(b!='')])
                    else:
                        bf_txt = noneText

                    r_dict['rackPrice'].append(rack_pr)
                    r_dict['price'].append(pr)
                    r_dict['breakfast'].append(bf_txt)

                    
                    price_blk_2 = price_block.findNext('td', class_='hprt-table-cell-price')

                    if(price_blk_2):
                        price_2 = price_blk_2.find(class_='hprt-price-price')
                        pr_2 = price_2.getText().replace('\n','')

                        rack_rate_price_2 = price_blk_2.find(class_='hp-blue-sans-rack-rate')
                        if(rack_rate_price_2):
                            rack_pr_2 = rack_rate_price_2.getText().replace('\n','')
                        else:
                            rack_pr_2 = pr_2

                        if(bf_icon):
                            bf_icon_2 = bf_icon.findNext(class_='hprt-conditions')
                            if(bf_icon_2):
                                bf2_list = bf_icon_2.getText().replace('\n',';').split(';')
                                bf2_txt = '; '.join([b for b in bf2_list if(b!='')])

                            r_dict['roomName'].append(room_name)
                            r_dict['capacity'].append(r_cap)
                            r_dict['bedType'].append(btype)
                            r_dict['features'].append(features)
                            r_dict['rackPrice'].append(rack_pr_2)
                            r_dict['price'].append(pr_2)
                            r_dict['breakfast'].append(bf2_txt)
                    
                else:
                    
                    r_dict['rackPrice'].append(noneText)
                    r_dict['price'].append(noneText)
                    r_dict['breakfast'].append(noneText)

    df_room = pd.DataFrame.from_dict(r_dict)

    return df_room

# Run all functions

In [17]:
df_all_hotel = pd.DataFrame()
df_all_room = pd.DataFrame()

with open('booking-urls.txt','r') as f:
    lines = [line.rstrip('\n') for line in f]
    for i in range(len(lines)):
        print(i)
        it += 1
        
        driver = webdriver.Chrome(home_path)
        driver.get(lines[i])
        raw_html = driver.execute_script("return document.documentElement.outerHTML")

        soup = BeautifulSoup(raw_html,'html.parser')

        df_hotel = get_hotel_table(soup, i)
        df_room = get_room_table(soup, i)

        # df_all_hotel = df_all_hotel.append(df_hotel)
        # df_all_room = df_all_room.append(df_room)# 
        
        if(not (os.path.isfile('all_hotel_booking.csv') and os.path.isfile('all_room_booking.csv'))):
            df_hotel.to_csv('all_hotel_booking.csv')
            df_room.to_csv('all_room_booking.csv')
            
        else:
            df_hotel.to_csv('all_hotel_booking.csv', mode='a', header=False)
            df_room.to_csv('all_room_booking.csv', mode='a', header=False)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

WebDriverException: Message: chrome not reachable
  (Driver info: chromedriver=72.0.3626.69 (3c16f8a135abc0d4da2dff33804db79b849a7c38),platform=Windows NT 10.0.17134 x86_64)


In [ ]:
# df_all_hotel.to_csv('all_hotel_booking.csv')
# df_all_hotel

In [ ]:
# df_all_room.to_csv('all_room_booking.csv')
# df_all_room

In [315]:
import sys

h = hash('Grand Business Inn') + sys.maxsize + 1
print(h)

8383728632389484888


In [316]:
h2 = hash('Grand Business Inn') + sys.maxsize + 1
print(h2)

8383728632389484888
